In [1]:
import sys
import os

import qgis
from qgis.gui import *
from qgis.core import *
from qgis.utils import plugins
from PyQt5.QtCore import *
from qgis.analysis import QgsNativeAlgorithms

sys.path.append('/usr/share/qgis/python/plugins/')
sys.path.append('/usr/share/qgis/python/')

QgsApplication.setPrefixPath('/usr', True)
app = QgsApplication([], False)
app.initQgis()

import processing
from processing.core.Processing import Processing
Processing.initialize()

QgsApplication.processingRegistry().addProvider(QgsNativeAlgorithms())
# for alg in QgsApplication.processingRegistry().algorithms():
#         print(alg.id(), "--->", alg.displayName())

# Stop QGIS appllication
# app.exitQgis()
# app.exit()

/usr/share/qgis/python/plugins/processing/algs/qgis/ImportIntoSpatialite.py:103: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if uri.database() is '':
/usr/share/qgis/python/plugins/processing/algs/qgis/SpatialiteExecuteSQL.py:69: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if uri.database() is '':
/usr/share/qgis/python/plugins/processing/algs/grass7/Grass7Utils.py:191: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if path is '':


True

In [2]:
from qgis.core import QgsProject, QgsApplication
# Get the project instance
project = QgsProject.instance()
# Print the current project file name (might be empty in case no projects have been loaded)
# print(project.fileName())

# Load another project
project.read(f"/home/luca/Code/mappy.git/input_data/qgis_project.qgz")
print(project.fileName())

/home/luca/Code/mappy.git/input_data/qgis_project.qgz


/usr/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
project.mapLayers()
cont = project.mapLayersByName("contacts")[0]
pols = project.mapLayersByName("map")[0]

In [4]:
index = QgsSpatialIndex() # Spatial index
for ft in pols.getFeatures():
    index.insertFeature(ft)


<ipython-input-4-01a47ce43a12>:3: DeprecationWarning: QgsSpatialIndex.insertFeature() is deprecated
  index.insertFeature(ft)


In [5]:
outtable = []

for fid, f in enumerate(cont.getFeatures()):
    g = f.geometry()
    if g.isEmpty():
        continue
    if g.isMultipart():
        for pid, p in enumerate(g.parts()):
            start = p[0]
            end = p[-1]
            
            outtable.append([fid, pid, [start, end]])
        


In [6]:
def find_inside(pt, index, polygons):
    ids = index.intersects(pt.boundingBox())
    
    pp= polygons.getFeatures(ids)
    real = []
    for p, id in zip(pp, ids):
        if p.geometry().contains(QgsPointXY(pt)):
            real.append(id)
            
    return real
            

    

In [7]:
cuttable = []
for fid, pid, (start, end) in outtable:
    real = find_inside(start, index, pols)
            
    if len(real) > 0:
        cuttable.append([fid, pid, real[0]])
        
    real = find_inside(end, index, pols)
            
    if len(real) > 0:
        cuttable.append([fid, pid, real[0]])

In [8]:
tocut = cuttable[0]

In [9]:
cuttable

[[32, 0, 10],
 [33, 0, 20],
 [34, 0, 10],
 [35, 0, 19],
 [36, 0, 30],
 [36, 0, 2],
 [37, 0, 29]]

In [10]:
l = cont.getFeature(36)
p = pols.getFeature(30)


In [11]:
cont.select(36)
pols.select(30)


In [12]:
pols.selectedFeatureIds()

[30]

In [13]:
pars = {"INPUT":QgsProcessingFeatureSourceDefinition(cont.name(), True), 
        "OVERLAY": QgsProcessingFeatureSourceDefinition(pols.name(), True),
        "OUTPUT":"output_file.shp"}

processing.run("native:difference",pars)

{'OUTPUT': 'output_file.shp'}

In [29]:
print([a.id() for a in QgsApplication.processingRegistry().algorithms() if "clip" in a.id()])

['gdal:cliprasterbyextent', 'gdal:cliprasterbymasklayer', 'gdal:clipvectorbyextent', 'gdal:clipvectorbypolygon', 'native:clip']


In [ ]:
[2021-02-15 11:39:59] processing.run("native:clip", {'INPUT':QgsProcessingFeatureSourceDefinition('contacts_7768adb1_5e8e_4a87_a7b2_aebd7bb4303e', True),
                                                     'OVERLAY':QgsProcessingFeatureSourceDefinition('map_62aaeba8_b51e_4242_82bc_ea14f8a87fbe', True),'OUTPUT':'TEMPORARY_OUTPUT'})